# Integrating different data sources into a unified database in MongoDB

The goal of this notebook is to build a simple document-based database on MongoDB by integrating multiple sources, in order to do some queries involving movies, ratings, and academy awards obtained.

The first source is the [Movielens 20M dataset](https://grouplens.org/datasets/movielens/20m/ "Main page of the Movielens 20M dataset"), which contains data about Movies and ratings that the users of Movielens provided for movies. It consists of several files, but we will use just some of them.

Another source is represented by the [Oscar Awards](https://www.kaggle.com/datasets/unanimad/the-oscar-award) dataset, which contains all the nominations at the Academy Awards ceremonies from 1927 to 2024.

Details on the detasets can be found on their respective pages.

---

**DISCLAIMER**

Notice that the goal of this notebook is to just show an example of a process involving some simple forms of data cleansing/integration, and some interactions with a target document-based database. So, all the steps have to be considered as "sketchy". You might still find anomalies arising from an incomplete ETL phase and you might think of better ways to model the source datasets.

## Retrieving the data sources

First of all, let's download the dataset, unzip them, and get rid of what we won't need.

**Note**: if you are working on a system which does not support Unix commands, you can just manually download and unzip the files through their respective URLs.

In [1]:
!curl -L https://files.grouplens.org/datasets/movielens/ml-20m.zip -o movielens20m.zip -C -
!unzip movielens20m.zip
!rm ml-20m/gen*.csv

!curl -L "https://drive.google.com/uc?export=download&id=1dBESl6rJYA72Obr8P8h64lejOhH50_Ua" -o academy_nominations.zip -C -
!unzip academy_nominations.zip

!mv the_oscar_award.csv ml-20m/academy_nominations.csv

!rm *.zip
!rm -rf *MACOS*

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  189M  100  189M    0     0  5177k      0  0:00:37  0:00:37 --:--:-- 6019k926k      0  0:01:06  0:00:05  0:01:01 3527k
Archive:  movielens20m.zip
   creating: ml-20m/
  inflating: ml-20m/genome-scores.csv  
  inflating: ml-20m/genome-tags.csv  
  inflating: ml-20m/links.csv        
  inflating: ml-20m/movies.csv       
  inflating: ml-20m/ratings.csv      
  inflating: ml-20m/README.txt       
  inflating: ml-20m/tags.csv         
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  202k  100  202k    0     0  58078      0  0:00:03  0:00:03 --:--:--   99k
Archive:  academy_nominations.zip
  inflating: the_oscar_award.csv     
  inflating: __

Let's import the needed libraries and have a look at the content of the files that we are going to use.

In [2]:
import pymongo
import pandas as pd
import re
from datetime import datetime

Let's create the dataframes of the data sources and let's display the content of each of them

In [3]:
folder = 'ml-20m/'
dataset = ['movies.csv', 'ratings.csv', 'tags.csv', 'links.csv', 'academy_nominations.csv']

sources = {}

for source in dataset:
    df = pd.read_csv(folder+source)
    pd.options.display.max_columns = len(df.columns)
    display(df.head(3))
    print(source)
    sources[source] = df

movieId                    title  \
0        1         Toy Story (1995)   
1        2           Jumanji (1995)   
2        3  Grumpier Old Men (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance

movies.csv


userId  movieId  rating   timestamp
0       1        2     3.5  1112486027
1       1       29     3.5  1112484676
2       1       32     3.5  1112484819

ratings.csv


userId  movieId          tag   timestamp
0      18     4141  Mark Waters  1240597180
1      65      208    dark hero  1368150078
2      65      353    dark hero  1368150079

tags.csv


movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0

links.csv


year_film  year_ceremony  ceremony category                 name  \
0       1927           1928         1    ACTOR  Richard Barthelmess   
1       1927           1928         1    ACTOR        Emil Jannings   
2       1927           1928         1  ACTRESS       Louise Dresser   

               film  winner  
0         The Noose   False  
1  The Last Command    True  
2   A Ship Comes In   False

academy_nominations.csv


First of all, we should decide which is the modeling that we want to adopt in our document-based DBMS.
The data model should be defined depending on the specific needs that the analysis phase poses (queries, denormalization, sharding, etc).

In our case, we will simply build a databased consisting of two collections: **movies** and **users**.

The aggregates (documents) that we want to keep in the **movies** collection should have the following "schema" \*
- *title*: title of the movie (string)
- *year*: year in which the movie has been released (int)
- *imdbid*: identifier used by IMDB (int) - retrieved from the *links.csv* source
- *genres*: array of strings, each of which representing one of the genres which the movie belongs to
- *tags*: array of strings representing tags assigned by Movielens users
- *nominations*: array of objects. Each object will be a document containing the field "*category*" (string identifying the nomination category), "*candidate*" (string representing the effective candidate, e.g., an actor, a movie studio, etc.), and "*win*" (boolean value indicating if the nomination was won).


The collection **users** will contain aggregates of this form:
- *userId*: identifier of users assigned by Movielens
- *reviews*: array of objects, where each object represents one review that the user provided, and it contains the fields: "*movie*" (int corresponding to the imdb id of the movie), "*ratings*" (double representing the rating that the user gave for that specific movie), and "*timestamp*" (value of type Date indicating when the review was provided).



\* We are not going to explicitly assess the presence of the *_id* field which is mandatory. MongoDB will automatically assign it to every document in the collection. We will rely on the ids provided by the data sources, btw.


As an example of how to use them, let's define the [schema validator](https://www.mongodb.com/docs/manual/core/schema-validation/specify-json-schema/) for the **users** collection that we will create later on.

In [4]:
users_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["reviews"],
        "properties": {
            "reviews": {
                "bsonType": "array",
                "items": {
                    "bsonType": "object",
                    "required": ["movie", "rating", "timestamp"],
                    "properties": {
                        "movie": {
                            "bsonType": "int",
                        },
                        "rating": {
                            "bsonType": "double",
                            "minimum": 0,
                            "maximum": 5
                        },
                        "timestamp": {
                            "bsonType": "date"
                        }
                    }
                }
            }
        }
    }
}

Let's set up the connection to the local Mongo DBMS instance.

Note: the service must be up and running.

In [5]:
conn = pymongo.MongoClient("mongodb://localhost:27017/")
print(conn.list_database_names())

database = conn["movielens"] # "creating" the movielens database
print(conn.list_database_names())

['admin', 'config', 'local']
['admin', 'config', 'local']


Notice that the database does not appear even after we instanciate it, since MongoDB actually creates the database only wen data is inserted into it.

Let's now create our first collection (movies).

In [6]:
col_movies = database["movies"]
print(database.list_collection_names())

[]


Again, we don't see any collection, since without any further insertion, the database still does not exist.

## Populating the movies collection

First of all, we notice that we have to clean our data through some ETL cleansing operations.
As an example, the *title* column in *movies.csv* contains both the titles and the years of movies. That's why we are going to define some function to tackle the problem of cleaning the data to fit our aggregate model.

In [7]:
def clean_title(s):
    pattern = r'(.+?)\s*\((\d+)\)'
    match = re.search(pattern, s)
    
    if match:
        substring = match.group(1)        
        number = int(match.group(2))       
        return substring, number
    else:
        return None, None

Since we are interested in the tags assigned to movies from the users, let's retrieve them from the tags dataframe.

In [8]:
s = sources['tags.csv']

tags = {}

for index, record in s.iterrows():
    if record['movieId'] not in tags:
        tags[record['movieId']] = set()
    tags[record['movieId']].add(str(record['tag']).lower())

In [9]:
print('Tags assigned to the movie whose id is 100:')
print(tags[100])

Tags assigned to the movie whose id is 100:
{'corruption', 'clv', 'harold becker', 'politics', 'al pacino', 'own'}


Let's retrieve, for each movie, the corresponding IMDB identifier.
Notice, the key of the dictionary is the movielens id of the movie, while the value is the IMDB id.

In [10]:
imdbids = {}

for index,record in sources['links.csv'].iterrows():
    imdbids[(int)(record['movieId'])] = (int)(record['imdbId'])

We are now ready to build the documents to insert into the **movies** collection.

In [11]:
s = sources['movies.csv']

movie = {}

for index, record in s.iterrows():
    title, year = clean_title(record['title']) # retrieve title and year
    movie['title'] = title
    movie['year'] = year
    movie['imdbid'] = imdbids[record['movieId']]
    movie['genres'] = []
    movie['tags'] = []
        
    for g in record['genres'].split('|'):
        movie['genres'].append(g)
    if record['movieId'] in tags:
        for t in tags[record['movieId']]:
            movie['tags'].append(t)
    
    col_movies.insert_one(movie) # at each iteration we add a new document to the collection
    movie = {}

Now there is a single collection in the database containing all the movies from the Movielens dataset.

At this point, we can create the collection of users as described before. This time, we will provide a validation schema for the collection.

In [12]:
col_users = database.create_collection("users", validator=users_validator)

Despite being empty, this time the collection has been created, due to the presence of data in the form of a validation schema.

Let's now integrate our database with the source about Academy Awards.
In particular, for each movie in the database, we want to include the set of academy nominations related to that movie. A movie is considered to be the same if both the title and the year coincide with those present in the database (notice that this might lead to forms of inconsistencies, since the data sources would require a much detailed cleaning).

In [13]:
# check if the pair title and year exists in our database
def exists_in_db(t,y):
    query = { "title": t, "year": y }
    return col_movies.count_documents(query)>0

In [14]:
# for each movie, add its nominations by updating the corresponding document through the $set operator
def add_nominations(nominations):
    for movie in nominations:
        for year in nominations[movie]:

            match_query = {"title": movie, "year": year }
            nomination_list = []
            for n in nominations[movie][year]:
                nomination_list.append({"category": n['category'],
                                "candidate": n['candidate'], 
                                "win": n['win']})
            query_update = {"$set": {"nominations": nomination_list}}
                          
            col_movies.update_many(match_query, query_update)

In [15]:
# we build a dictionary containing the nominations for every movie and we add them to the database
s = sources['academy_nominations.csv']

nominations = {}

for index, record in s.iterrows():
    nominee = {}
    title = record['film']
    year = record['year_film']
    if exists_in_db(title, year):
        nominee['category'] = record['category'].capitalize()
        nominee['candidate'] = record['name']
        nominee['win'] = bool(record['winner'])
        
        if title not in nominations:
            nominations[title] = {}
            nominations[title][year] = []
        else:
            if year not in nominations[title]:
                nominations[title][year] = []
                
        nominations[title][year].append(nominee)
        
add_nominations(nominations)

At this point we populate the **users** collection. To do so, we are going to build the aggregates representing our documents.

In [16]:
# we build a dictionary with a key for each user and, as value, an array of objects representing their reviews
users_docs = {}

for index, record in sources['ratings.csv'].iterrows():
    uid = (int)(record['userId'])
    imdb = (int)(imdbids[record['movieId']])
    rating = round(float(record['rating']), 1)
    tstamp = datetime.utcfromtimestamp(record['timestamp'])
    
    if uid not in users_docs:
        users_docs[uid] = []
            
    rating = {"movie": imdb, "rating": rating, "timestamp": tstamp}
    
    users_docs[uid].append(rating)
    


We can finally proceed to the insertion of the users in the database.

In [17]:
for u in users_docs:
    user = {"userId": u, "reviews": users_docs[u]}
    col_users.insert_one(user)

## Querying the database

At this point we can query the database by executing **find** operations or **aggregation pipelines**

### Query 1
Find all movies that won at least one Oscars between 2014 and 2024

In [18]:
query1 = {
    "year": {"$gte": 2010, "$lte": 2015}, 
    "nominations.win": {"$eq": True}}

projection = {
    "_id": 0,
    "title": 1,
    "year": 1,
    "nominations.category": 1
}

In [19]:
results = list(col_movies.find(query1, projection))

In [20]:
for d in results:
    print(d,'\n')

{'title': 'Alice in Wonderland', 'year': 2010, 'nominations': [{'category': 'Art direction'}, {'category': 'Costume design'}, {'category': 'Visual effects'}]} 

{'title': 'Toy Story 3', 'year': 2010, 'nominations': [{'category': 'Animated feature film'}, {'category': 'Music (original song)'}, {'category': 'Best picture'}, {'category': 'Sound editing'}, {'category': 'Writing (adapted screenplay)'}]} 

{'title': 'Inception', 'year': 2010, 'nominations': [{'category': 'Art direction'}, {'category': 'Cinematography'}, {'category': 'Music (original score)'}, {'category': 'Best picture'}, {'category': 'Sound editing'}, {'category': 'Sound mixing'}, {'category': 'Visual effects'}, {'category': 'Writing (original screenplay)'}]} 

{'title': 'Inside Job', 'year': 2010, 'nominations': [{'category': 'Documentary (feature)'}]} 

{'title': 'Black Swan', 'year': 2010, 'nominations': [{'category': 'Actress in a leading role'}, {'category': 'Cinematography'}, {'category': 'Directing'}, {'category': 'F

### Pipeline 1 - Average rating per user
For each user in the *users* collection, we want to compute the average rating that such user provided for the movies (s)he reviewed, and we want to sort users by average rating in descending order.

In [21]:
pipeline1 = [
  {
    "$unwind": "$reviews", # create a new document for each review in the reviews array
  },
  {
    "$group": {
      "_id": "$userId", # group by userId
      "averageRating": { # compute the new field averageRating
        "$avg": "$reviews.rating", # as the average of the ratings of the given user
      },
    },
  },
  {
    "$sort": {
      "averageRating": -1, # sort in descending order
    },
  },
]

In [22]:
results = list(col_users.aggregate(pipeline1))

In [23]:
for d in results:
    print(d)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'_id': 102872, 'averageRating': 3.0}
{'_id': 32995, 'averageRating': 3.0}
{'_id': 113598, 'averageRating': 3.0}
{'_id': 91242, 'averageRating': 3.0}
{'_id': 87041, 'averageRating': 3.0}
{'_id': 38514, 'averageRating': 3.0}
{'_id': 35606, 'averageRating': 3.0}
{'_id': 40888, 'averageRating': 3.0}
{'_id': 101366, 'averageRating': 3.0}
{'_id': 133561, 'averageRating': 3.0}
{'_id': 77957, 'averageRating': 3.0}
{'_id': 62893, 'averageRating': 3.0}
{'_id': 91342, 'averageRating': 3.0}
{'_id': 98083, 'averageRating': 3.0}
{'_id': 27750, 'averageRating': 3.0}
{'_id': 48385, 'averageRating': 3.0}
{'_id': 6954, 'averageRating': 3.0}
{'_id': 104025, 'averageRating': 3.0}
{'_id': 18468, 'averageRating': 3.0}
{'_id': 27706, 'averageRating': 3.0}
{'_id': 83745, 'averageRating': 3.0}
{'_id': 41717, 'averageRating': 3.0}
{'_id': 89019, 'averageRating': 3.0}
{'_id': 72585, 'averageRating': 3.0}
{'_id': 57085, 'averageRating': 3.0}
{'_id': 101643, 'averageRating': 3.0}
{'_id': 115573, 'averageRating': 

### Pipeline 2 - Average rating per each genre assigned by the most active reviewer
Given the user with the highest number of reviews, we want to compute the average rating that such user assigned for every different genre.

In [ ]:
pipeline2 = [
  {
    "$addFields": {
      "totReviews": {
        "$size": "$reviews",
      },
    },
  },
  {
    "$sort": {
      "totReviews": -1,
    },
  },
  {
    "$limit": 1,
  },
  {
    "$unwind": "$reviews",
  },
  {
    "$lookup": {
      "from": "movies",
      "localField": "reviews.movie",
      "foreignField": "imdbid",
      "as": "movieDetails",
    },
  },

  {
    "$sort": {
      "reviews.rating": -1,
    },
  },

  {
    "$addFields": {
      "movieTitle": {
        "$arrayElemAt": ["$movieDetails.title", 0],
      },
      "movieGenres": {
        "$arrayElemAt": ["$movieDetails.genres", 0],
      },
    },
  },

  {
    "$unwind": "$movieGenres",
  },

  {
    "$group": {
      "_id": "$movieGenres",
      "avgRating": {
        "$avg": "$reviews.rating",
      },
    },
  },
    {
    "$addFields": {
      "roundAvgRating": {
        "$round": ["$avgRating", 2],
      },
    },
  },
    {
        "$project": {
            "_id": 1,
            "roundAvgRating": 1
        }
    },

  {
    "$sort": {
      "roundAvgRating": -1,
    },
  },
]

In [25]:
results = list(col_users.aggregate(pipeline2))

In [26]:
for d in results:
    print(d)

{'_id': 'IMAX', 'roundAvgRating': 3.71}
{'_id': 'Documentary', 'roundAvgRating': 3.45}
{'_id': 'Animation', 'roundAvgRating': 3.44}
{'_id': 'War', 'roundAvgRating': 3.43}
{'_id': 'Film-Noir', 'roundAvgRating': 3.42}
{'_id': 'Drama', 'roundAvgRating': 3.38}
{'_id': 'Fantasy', 'roundAvgRating': 3.32}
{'_id': 'Crime', 'roundAvgRating': 3.32}
{'_id': 'Romance', 'roundAvgRating': 3.32}
{'_id': 'Mystery', 'roundAvgRating': 3.29}
{'_id': 'Adventure', 'roundAvgRating': 3.28}
{'_id': 'Thriller', 'roundAvgRating': 3.28}
{'_id': 'Children', 'roundAvgRating': 3.27}
{'_id': 'Musical', 'roundAvgRating': 3.23}
{'_id': 'Action', 'roundAvgRating': 3.22}
{'_id': 'Western', 'roundAvgRating': 3.22}
{'_id': 'Comedy', 'roundAvgRating': 3.2}
{'_id': 'Sci-Fi', 'roundAvgRating': 3.06}
{'_id': 'Horror', 'roundAvgRating': 2.94}


### Pipeline 3 - Most popular genres
Find the most common genre among the documents in the movies collection.

In [27]:
pipeline3 = [
  {
    "$unwind": "$genres",
  },
  {
    "$group": {
      "_id": "$genres",
      "count": {
        "$sum": 1,
      },
    },
  },
  {
    "$sort": {
      "count": -1,
    },
  },
]

In [28]:
results = list(col_movies.aggregate(pipeline3))

In [29]:
for d in results:
    print(d,'\n')

{'_id': 'Drama', 'count': 13344} 

{'_id': 'Comedy', 'count': 8374} 

{'_id': 'Thriller', 'count': 4178} 

{'_id': 'Romance', 'count': 4127} 

{'_id': 'Action', 'count': 3520} 

{'_id': 'Crime', 'count': 2939} 

{'_id': 'Horror', 'count': 2611} 

{'_id': 'Documentary', 'count': 2471} 

{'_id': 'Adventure', 'count': 2329} 

{'_id': 'Sci-Fi', 'count': 1743} 

{'_id': 'Mystery', 'count': 1514} 

{'_id': 'Fantasy', 'count': 1412} 

{'_id': 'War', 'count': 1194} 

{'_id': 'Children', 'count': 1139} 

{'_id': 'Musical', 'count': 1036} 

{'_id': 'Animation', 'count': 1027} 

{'_id': 'Western', 'count': 676} 

{'_id': 'Film-Noir', 'count': 330} 

{'_id': '(no genres listed)', 'count': 246} 

{'_id': 'IMAX', 'count': 196} 



### Pipeline 4 (Challenge)
For each movie in the movies collection, we want to compute the average rating that it got from all the reviews it received.

While this query might not seem particularly challenging, the data model that we adopted makes it quite hard for the system to execute it.
The reason is that, for each movie, we have to check all the objects in the reviews arrays for all the users in the user collection in order to check if they rated the movie and to retrieve the rating that the movie got.

As an exercise, you can try to build an aggregation pipeline to solve this problem. You might try to create indexes for specific fields if you think the system would benefit from them.